In [12]:
#라이브러리 import 및 설정
import lightgbm as lgb
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
from sklearn.model_selection import  train_test_split

#데이터 로드
data_dir = Path('C:\\Users\\USER\\Desktop\\경진대회_dataset\\DataInput')
sub_dir = Path('C:\\Users\\USER\\Desktop\\경진대회_dataset\\sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

seed =150 #적정값이 있었음 너무 낮거나 높음 떨어짐
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

sub_file = sub_dir / f'{model_name}.csv'

df = pd.read_csv(trn_file, header=None)
print(df.shape)
df.head()

(4280, 32)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,HI,188,128,95,114,143,108,88,103,113,...,100,78,70,79,84,66,70,75,76,63
1,PH,174,112,88,104,119,92,74,79,88,...,82,65,70,75,89,73,67,71,89,73
2,GR,175,138,106,105,135,109,75,95,113,...,110,98,67,88,119,98,75,91,110,94
3,PH,176,111,80,106,131,96,76,99,104,...,96,78,78,91,96,78,82,104,112,85
4,EL,182,144,111,100,151,119,67,106,114,...,108,88,71,103,113,92,68,107,118,92


In [13]:
df_tst= pd.read_csv(tst_file, header=None)
print(df_tst.shape)
df_tst.head()

(1833, 31)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,185,130,100,119,142,110,89,102,114,87,...,100,81,84,103,108,88,88,112,113,88
1,178,139,100,104,142,114,63,97,114,94,...,108,88,60,95,113,92,53,95,108,88
2,182,125,97,112,137,105,78,97,105,83,...,108,85,84,103,108,85,80,99,104,81
3,174,113,83,97,115,89,63,63,82,69,...,83,59,64,68,75,63,64,68,83,67
4,167,144,107,97,146,124,67,111,114,101,...,122,99,68,107,128,96,71,112,128,99


In [14]:
y = df.iloc[:,0].values[:4280]       #종속
df.drop(df.columns[[0]], axis=1, inplace=True)  #독립을 위한 종속 제거
trn = df.iloc[:4280].values 
tst=df_tst.iloc[:1834,:32].values              #train을 위한 독립 320000개     
print(y.shape, trn.shape, tst.shape)

#학습/검증 데이터 구분_[Hold-out Validation]
#>학습 후 모델의 예측/분류 정확도 계산을 위한 검증데이터 분리
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)
params = {
    "objective": "multiclass",
  #  "n_estimators": 1000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1
}
#고정 파라미터

space = {
    "boosting_type":hp.choice("boosting_type", ['gbdt', 'dart', 'rf']),
    "n_estimators":  hp.choice("n_estimators", [100, 200, 300, 400, 500, 600,800,1000]),
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.1)),
   # "num_iterations":  hp.choice("num_iterations", [100, 200, 300, 400, 500, 600,800,1000]),
    "num_leaves": hp.choice("num_leaves", [10, 15, 31, 63, 127, 254]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1)
}
#변경 파라미터_넓은 범위와 여러 파라미터 사용시 시간 급증 (주의)
#loguniform: 로그함수를 이용해 확률 분포 내에서의 값으로 test
#choice: 리스트 값 중 하나로 선택하여 test
#quniform: start와 stop사이에서 일정 간격으로 값을 주어 test


(4280,) (4280, 31) (1833, 31)


In [15]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",#멀티클래스_로그 손실함수
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]
    #측정한 파라미터 중단됐을 때 최고의 성능점수

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials, algo=tpe.suggest, max_evals=10, verbose=1)
# fn=최소화하고 싶은 함수와 파라미터(space), 
# trials=실행 객체변수, tpe.sugges=알고리즘 선택
# max_evals=서치 횟수 

 10%|████▊                                           | 1/10 [00:00<00:03,  2.48trial/s, best loss: 0.22622902762859975]

C:\Users\USER\anaconda3\lib\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



 20%|█████████▌                                      | 2/10 [01:15<03:01, 22.70s/trial, best loss: 0.22622902762859975]

C:\Users\USER\anaconda3\lib\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



 40%|███████████████████▏                            | 4/10 [01:17<01:09, 11.62s/trial, best loss: 0.22622902762859975]

...

 70%|█████████████████████████████████▌              | 7/10 [01:20<00:13,  4.59s/trial, best loss: 0.22622902762859975]

C:\Users\USER\anaconda3\lib\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



 80%|██████████████████████████████████████▍         | 8/10 [01:47<00:22, 11.44s/trial, best loss: 0.22622902762859975]

C:\Users\USER\anaconda3\lib\site-packages\lightgbm\callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



100%|███████████████████████████████████████████████| 10/10 [02:17<00:00, 13.75s/trial, best loss: 0.22622902762859975]


In [16]:
hyperparams = space_eval(space, best)   #검색하여 나온 최적의 파라미터
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)                           #업데이트된 파라미터

{'objective': 'multiclass', 'subsample_freq': 1, 'random_state': 150, 'n_jobs': -1, 'boosting_type': 'gbdt', 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.06316437855450058, 'n_estimators': 600, 'num_leaves': 10, 'subsample': 0.6000000000000001}


{'objective': 'multiclass', 'subsample_freq': 1, 'random_state': 150, 'n_jobs': -1,   
'boosting_type': 'dart', 'colsample_bytree': 0.6000000000000001,   
'learning_rate': 0.028778270063003097, 'n_estimators': 100, 'num_iterations': 1000,  
'num_leaves': 31, 'subsample': 0.6000000000000001}

{'objective': 'multiclass', 'subsample_freq': 1, 'random_state': 150,  
'n_jobs': -1, 'boosting_type': 'gbdt', 'colsample_bytree': 0.7000000000000001,   
'learning_rate': 0.06316437855450058, 'n_estimators': 600, 'num_leaves': 10,  
'subsample': 0.6000000000000001}


In [17]:

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
   
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold
#91.8224% 7 90.8879% 14  90.8879% 50   91.2383% 100  91.4720% 107,130   91.2409% 114  90.3650% 174 89.8364%289
# n_estimators=1000,learning_rate=0.01, 92.0561%(num_leaves=10)_overfit 91.7056%
#num_leaves 91.7056% 16 91.9393% 32 91.9393% 64 (100,0.1)
#90.3037%
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

NameError: name 'StratifiedKFold' is not defined

In [ ]:
print(len(p_tst))
imp = pd.DataFrame({'feature': trn.columns, 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')

In [ ]:
sub = pd.read_csv(sample_file)
sub['class'] = p_tst
sub.to_csv(sub_file)